In [ ]:
import joblib
from tqdm import tqdm_notebook
import pandas as pd
import re

## preprocess
3回に分けて取得したツイートの結合

In [ ]:
tweets1 = joblib.load('../data/tweet_212551.pkl')
tweets2 = joblib.load('../data/tweet_085225.pkl')
tweets3 = joblib.load('../data/tweet_193053.pkl')

In [ ]:
pokemon1 = pd.io.json.json_normalize(tweets1)
pokemon2 = pd.io.json.json_normalize(tweets2)
pokemon3 = pd.io.json.json_normalize(tweets3)

In [ ]:
pokemon = pd.concat([pokemon1, pokemon2, pokemon3], ignore_index=True, sort=False, axis=0)

In [ ]:
pokemon.shape

In [ ]:
# 秒数を重複するように取得したので重複行を削除
pokemon = pokemon.drop_duplicates()

In [ ]:
pokemon.shape

In [ ]:
pokemon.to_csv('../data/pokemon.csv.gz', compression='gzip')

選ばれたポケモンの抽出

In [ ]:
def extract_pokemon(x):
    try:
        x = x.replace('\n', '')
        return re.search('キミは【(.*)】の要素を持つ人材', x).groups()[0]
    except AttributeError:
        return None

In [ ]:
pokemon['pokemon'] = pokemon.text.apply(extract_pokemon)

In [ ]:
pokemon.pokemon.value_counts()[:20].to_csv('../data/ranking.csv')

時系列推移

In [ ]:
pokemons = pokemon.pokemon.value_counts()[:20].index.tolist()

In [ ]:
pokemon = pokemon[pokemon.pokemon.isin(pokemons)]

In [ ]:
hourly_df = pokemon[['created_at', 'pokemon']].copy()

In [ ]:
hourly_df.created_at = hourly_df.created_at.dt.tz_localize('Asia/Tokyo')

In [ ]:
hourly_df['value'] = 1

In [ ]:
pd.pivot_table(hourly_df, index=pd.Grouper(key='created_at', freq='1h'),
               columns='pokemon', values='value', aggfunc=sum).fillna(0).T.to_csv('../data/timeseries.csv', encoding='utf8')

## TODO
WIP

In [ ]:
import tweepy, configparser

In [ ]:
conf = configparser.ConfigParser()
conf.read('../config/setting.ini')

In [ ]:
auth = tweepy.OAuthHandler(conf['twitterapi']['consumer_key'], conf['twitterapi']['consumer_secret'])
auth.set_access_token(conf['twitterapi']['token'], conf['twitterapi']['token_secret'])

In [ ]:
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
friends_ids = []
my_info = api.me()

for friend_id in tweepy.Cursor(api.friends_ids, user_id=my_info.id).items():
    friends_ids.append(friend_id)


In [ ]:
len(friends_ids)

In [ ]:
pokemon[pokemon['user.id'].isin(friends_ids)].pokemon.value_counts()